In [1]:
import os
import tempfile
import warnings
from os.path import join as pjoin

import dask
import dask.dataframe as dd
import dask_geopandas as dgpd
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
from scipy.fft import dst
import tqdm
import xarray as xr
from dask.diagnostics import ProgressBar
from rasterio.crs import CRS

from raster_tools import Raster, Vector, open_vectors, clipping, zonal
from raster_tools.dtypes import F32, U8, U16

import matplotlib.pyplot as plt

# Location for temporary storage
TMP_LOC = "/home/jake/FireLab/Project/data/temp/"
DATA_LOC = "/home/jake/FireLab/Project/data/"

STATE = "OR"

# Location of clipped DEM files
DEM_DATA_DIR = pjoin(TMP_LOC, "dem_data")

# location of feature data files
FEATURE_DIR = pjoin(DATA_LOC, "FeatureData")
EDNA_DIR = pjoin(DATA_LOC, "terrain")
MTBS_DIR = pjoin(DATA_LOC, "MTBS_Data")

mtbs_df_path = pjoin(TMP_LOC, f"{STATE}_mtbs.parquet/")
mtbs_df_temp_path = pjoin(TMP_LOC, f"{STATE}_mtbs_temp.parquet/")
checkpoint_1_path = pjoin(TMP_LOC, "check1")
checkpoint_2_path = pjoin(TMP_LOC, "check2")

PATHS = {
    "states": pjoin(EDNA_DIR, "state_borders/cb_2018_us_state_5m.shp"),
    "dem": pjoin(EDNA_DIR, "us_orig_dem/us_orig_dem/orig_dem/hdr.adf"),
    "dem_slope": pjoin(EDNA_DIR, "us_slope/us_slope/slope/hdr.adf"),
    "dem_aspect": pjoin(EDNA_DIR, "us_aspect/aspect/hdr.adf"),
    "dem_flow_acc": pjoin(EDNA_DIR, "us_flow_acc/us_flow_acc/flow_acc/hdr.adf"),
    "gm_srad": pjoin(FEATURE_DIR, "gridmet/srad_1986_2020_weekly.nc"),
    "gm_vpd": pjoin(FEATURE_DIR, "gridmet/vpd_1986_2020_weekly.nc"),
    "aw_mat": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MAT.tif"),
    "aw_mcmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MCMT.tif"),
    "aw_mwmt": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_MWMT.tif"),
    "aw_td": pjoin(FEATURE_DIR, "adaptwest/Normal_1991_2020_TD.tif"),
    "dm_tmax": pjoin(FEATURE_DIR, "daymet/tmax_1986_2020.nc"),
    "dm_tmin": pjoin(FEATURE_DIR, "daymet/tmin_1986_2020.nc"),
    "biomass_afg": pjoin(
        FEATURE_DIR, "biomass/biomass_afg_1986_2020_{}.nc".format(STATE)
    ),
    "biomass_pfg": pjoin(
        FEATURE_DIR, "biomass/biomass_pfg_1986_2020_{}.nc".format(STATE)
    ),
    "landfire_fvt": pjoin(
        FEATURE_DIR, "landfire/LF2020_FVT_200_CONUS/Tif/LC20_FVT_200.tif"
    ),
    "landfire_fbfm40": pjoin(
        FEATURE_DIR, "landfire/LF2020_FBFM40_200_CONUS/Tif/LC20_F40_200.tif"
    ),
    "ndvi": pjoin(FEATURE_DIR, "ndvi/access/weekly/ndvi_1986_2020_weekavg.nc"),
    "mtbs_root": pjoin(MTBS_DIR, "MTBS_BSmosaics/"),
    "mtbs_perim": pjoin(MTBS_DIR, "mtbs_perimeter_data/mtbs_perims_DD.shp"),
}
YEARS = list(range(2016, 2021))
GM_KEYS = list(filter(lambda x: x.startswith("gm_"), PATHS))
AW_KEYS = list(filter(lambda x: x.startswith("aw_"), PATHS))
DM_KEYS = list(filter(lambda x: x.startswith("dm_"), PATHS))
BIOMASS_KEYS = list(filter(lambda x: x.startswith("biomass_"), PATHS))
LANDFIRE_KEYS = list(filter(lambda x: x.startswith("landfire_"), PATHS))
NDVI_KEYS = list(filter(lambda x: x.startswith("ndvi"), PATHS))
DEM_KEYS = list(filter(lambda x: x.startswith("dem"), PATHS))

In [6]:
stdf = open_vectors(PATHS["states"], 0).data.to_crs("EPSG:5071")
states = {st: stdf[stdf.STUSPS == st].geometry for st in list(stdf.STUSPS)}
state_shape = states[STATE]
states = None
stdf = None
perimdf = open_vectors(PATHS["mtbs_perim"]).data.to_crs("EPSG:5071")
state_fire_perims = perimdf.clip(state_shape.compute())
state_fire_perims = (
    state_fire_perims.assign(
        Ig_Date=lambda frame: dd.to_datetime(
            frame.Ig_Date, format="%Y-%m-%d"
        )
    )
    .sort_values("Ig_Date")
    .compute()
)
year_to_perims = {
    y: state_fire_perims[state_fire_perims.Ig_Date.dt.year == y]
    for y in YEARS
}
state_fire_perims = None
year_to_mtbs_file = {
    y: pjoin(PATHS["mtbs_root"], f"mtbs_{STATE}_{y}.tif")
    for y in YEARS
}

In [17]:
year_to_mtbs_file

{2016: '/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2016.tif',
 2017: '/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2017.tif',
 2018: '/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2018.tif',
 2019: '/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2019.tif',
 2020: '/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2020.tif'}

In [16]:
pd.DataFrame(year_to_perims[2016])

,Event_ID,irwinID,Incid_Name,Incid_Type,Map_ID,Map_Prog,Asmnt_Type,BurnBndAc,BurnBndLat,BurnBndLon,...,Perim_ID,dNBR_offst,dNBR_stdDv,NoData_T,IncGreen_T,Low_T,Mod_T,High_T,Comment,geometry
24769,OR4473012140020160415,None,2016 AERIAL MAINTENANCE UNDERBURNING,Prescribed Fire,10005251,MTBS,Initial,4200,44.645,-121.424,...,None,10,18,-970,-150,35,9999,9999,None,"POLYGON ((-1990379.301 2675943.793, -1990328.2..."
24710,OR4372011979020160515,None,UNNAMED,Prescribed Fire,10004895,MTBS,Extended,2095,43.742,-119.783,...,None,13,30,-970,-150,50,249,480,None,"POLYGON ((-1889049.402 2544287.648, -1889098.0..."
24045,OR4278111760620160606,516D8A8A-7BA6-49B3-BAB9-46553EE5400B,OWYHEE CANYON,Wildfire,26479,MTBS,Initial,22385,42.746,-117.506,...,None,-23,14,-970,-150,70,400,9999,None,"POLYGON ((-1741323.761 2388617.197, -1741317.4..."
24768,OR4448812148120160607,5BCA6540-74D4-40C9-BAE2-2C6F16002BCD,0152 AKAWANA OD,Wildfire,10005236,MTBS,Extended,2113,44.498,-121.45,...,None,-41,11,-970,-150,40,290,605,None,"POLYGON ((-1994246.224 2659687.220, -1994196.4..."
24026,OR4536912057620160608,0E5DE90B-3D42-4E92-9B9D-1BD3C846C651,0162 RN CURRIE CANYON,Wildfire,26444,MTBS,Initial,6338,45.36,-120.571,...,None,32,21,-970,-150,70,360,9999,None,"POLYGON ((-1901927.365 2731571.329, -1901978.1..."
24024,OR4562412106720160701,22B38812-1C54-4830-964C-7EB9F232FFFD,FIFTEEN MILE,Wildfire,26441,MTBS,Initial,4044,45.638,-121.006,...,None,32,32,-970,-150,60,400,9999,None,"POLYGON ((-1922441.100 2770163.472, -1922491.9..."
24028,OR4552312036720160721,7C1F653A-5CB5-465C-B71A-E9B8F337B0FD,0353 RN SCOTT CANYON,Wildfire,26447,MTBS,Initial,32051,45.442,-120.289,...,None,-17,6,-970,-150,50,280,9999,None,"POLYGON ((-1872891.230 2736121.964, -1872851.5..."
24044,OR4486912109820160724,72D45EA0-1017-4973-B160-7415D4277258,RATTLESNAKE,Wildfire,26474,MTBS,Initial,9296,44.835,-121.118,...,None,-15,8,-970,-150,10,250,9999,None,"POLYGON ((-1957973.533 2685957.735, -1958011.6..."
24694,OR4292412221220160728,0785ED60-034B-4C07-8F76-7B1C6A23FF7C,BYBEE CREEK,Wildfire,10004847,MTBS,Extended,1007,42.924,-122.195,...,804503020160805,-17,22,-970,-150,10,180,380,None,"POLYGON ((-2099102.662 2506285.741, -2099077.2..."
24775,OR4440811838320160731,60FA077F-D08E-4F51-A0A7-B8DF892AA977,RAIL,Wildfire,10005286,MTBS,Extended,43813,44.348,-118.347,...,None,-21,25,-970,-150,50,216,420,Used GeoMAC perimeter.,"POLYGON ((-1759563.961 2592778.091, -1759524.8..."


In [18]:
mtbs_2016 = year_to_mtbs_file[2016]
mtbs_2016

'/home/jake/FireLab/Project/data/MTBS_Data/MTBS_BSmosaics/mtbs_OR_2016.tif'

In [19]:
ifp = mtbs_2016
raster = rasterio.open(ifp)
print("raster.crs", raster.crs)

raster.crs ESRI:102039
